In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


plt.style.use('fivethirtyeight')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [3]:
rv_final = pd.read_csv('reviews_final.csv')
rv_final.head()

,db_id,drug,condition,review,rating
0,DB00002,['erbitux' 'cetuximab'],['colorectal cance' 'head and neck cance' 'squ...,"['""i have stage 4 colon cancer with liver mets...",6.875000
1,DB00003,['dornase alfa' 'pulmozyme'],['cystic fibrosis'],"['""my outcome/experience with pulmozyme was go...",10.000000
2,DB00005,['etanercept'],['rheumatoid arthritis' 'psoriatic arthritis' ...,"['""took enbrel with methotrexate for four yea...",8.246305
3,DB00007,['leuprolide' 'lupron depot-ped' 'lupron' 'eli...,['uterine fibroids' 'prostate cance' 'endometr...,"['""i was so skeptical about these shots, scare...",6.179487
4,DB00008,['pegasys' 'peginterferon alfa-2a'],['hepatitis c'],"['""i was treated with this twice on it&#039;s ...",6.791667


In [4]:
rv_final['n_reviews'] = rv_final.apply(lambda x: len([i for i in x['review'].split('"') if len(i) > 4]), axis = 1)
rv_final['condition'] = rv_final['condition'].str.replace('cance', 'cancer')
rv_final['condition'] = rv_final['condition'].str.replace('disorde', 'disorder')

rv_final.head()

,db_id,drug,condition,review,rating,n_reviews
0,DB00002,['erbitux' 'cetuximab'],['colorectal cancer' 'head and neck cancer' 's...,"['""i have stage 4 colon cancer with liver mets...",6.875000,16
1,DB00003,['dornase alfa' 'pulmozyme'],['cystic fibrosis'],"['""my outcome/experience with pulmozyme was go...",10.000000,2
2,DB00005,['etanercept'],['rheumatoid arthritis' 'psoriatic arthritis' ...,"['""took enbrel with methotrexate for four yea...",8.246305,203
3,DB00007,['leuprolide' 'lupron depot-ped' 'lupron' 'eli...,['uterine fibroids' 'prostate cancer' 'endomet...,"['""i was so skeptical about these shots, scare...",6.179487,273
4,DB00008,['pegasys' 'peginterferon alfa-2a'],['hepatitis c'],"['""i was treated with this twice on it&#039;s ...",6.791667,24


In [5]:
rv_final.shape

(1184, 6)

In [24]:
rv_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1184 entries, 0 to 1183
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   db_id      1184 non-null   object 
 1   drug       1184 non-null   object 
 2   condition  1184 non-null   object 
 3   review     1184 non-null   object 
 4   rating     1184 non-null   float64
 5   n_reviews  1184 non-null   int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 55.6+ KB


In [6]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [7]:
tvec = TfidfVectorizer(stop_words = 'english')
stopwords_condition = list(tvec.get_stop_words())
stopwords_condition.extend(['nan', 'listed', 'othe'])

In [48]:
tvec1 = TfidfVectorizer(stop_words = stopwords_condition,
                       max_features = 10000,
                       strip_accents = 'ascii',
                       lowercase = True,
                       decode_error = 'replace',
                       ngram_range = (2, 3),
                       min_df = 0.001)

sparse1 = tvec1.fit_transform(rv_final['condition'].values.astype(str))

In [49]:
# convert to df
tvec1_df = pd.DataFrame(sparse1.toarray(),
                       columns = tvec1.get_feature_names())

tvec1_df.head()

,abnormal uterine,abnormal uterine bleeding,acial wrinkles,acid deficiency,acne abnormal,acne abnormal uterine,acne bacterial,acne bacterial skin,acne birth,acne birth control,...,work sleep disorder,xerostomia mucositis,yeast infection,yeast infection onychomycosis,yeast infection tinea,zen shoulde,zinc oxide,zoste herpes,zoste herpes simplex,zoster prophylaxis
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
tvec1_df.sum(axis = 0).sort_values(ascending = False)[:20]

diabetes type              24.535388
high blood                 19.495993
blood pressure             19.495993
high blood pressure        19.495993
dietary supplementation    17.727156
birth control              16.543045
rheumatoid arthritis       15.275418
hiv infection              13.896807
multiple sclerosis         13.774915
nausea vomiting            13.213466
allergic rhinitis          12.861713
herbal supplementation     12.020522
nasal congestion           10.417535
tract infection            10.195330
asthma maintenance          9.830026
urinary tract               9.749393
head lice                   8.653370
bowel preparation           8.622065
urinary tract infection     8.368161
parkinson disease           8.116304
dtype: float64

In [8]:
cvec = CountVectorizer(stop_words = stopwords_condition,
                       max_features = 10000,
                       strip_accents = 'unicode',
                       lowercase = True,
                       decode_error = 'replace',
                       ngram_range = (2, 3),
                       min_df = 0.001)

# get sparse matrix
sparse = cvec.fit_transform(rv_final['condition'].values.astype(str))

In [9]:
# convert to df
cvec_df = pd.DataFrame(sparse.toarray(),
                       columns = cvec.get_feature_names())

cvec_df.head()

,abnormal uterine,abnormal uterine bleeding,acial wrinkles,acid deficiency,acne abnormal,acne abnormal uterine,acne bacterial,acne bacterial skin,acne birth,acne birth control,...,work sleep disorder,xerostomia mucositis,yeast infection,yeast infection onychomycosis,yeast infection tinea,zen shoulde,zinc oxide,zoste herpes,zoste herpes simplex,zoster prophylaxis
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
# get most frequent words
top_conditions_db = cvec_df.sum(axis = 0).sort_values(ascending = False)
top_conditions_db[:20]

influenza prophylaxis        245
nausea vomiting              119
birth control                 97
vomiting pregnancy            83
nausea vomiting pregnancy     83
blood pressure                76
high blood pressure           76
high blood                    76
rheumatoid arthritis          73
diabetes type                 59
tract infection               58
allergic rhinitis             49
nasal congestion              46
uterine bleeding              46
abnormal uterine              45
abnormal uterine bleeding     45
urinary tract                 44
urinary tract infection       39
crohn disease                 37
bipolar disorder              35
dtype: int64

In [19]:
top_conditions_db = pd.DataFrame(top_conditions_db)
top_conditions_db.columns = ['cvec_count']
top_conditions_db.head()

,cvec_count
influenza prophylaxis,245
nausea vomiting,119
birth control,97
vomiting pregnancy,83
nausea vomiting pregnancy,83


In [20]:
# top_conditions_db.to_csv('nlp_top_conditions.csv')

In [11]:
# remove numbers for tvec
rv_final.review = rv_final.review.str.replace(r'\d','')

In [12]:
stopwords_review = list(tvec.get_stop_words())
stopwords_review.extend(['ve', 'mg', 'like', 'did', 'don'])

In [13]:
# use stemmer inside tvec

# from nltk.stem import PorterStemmer
# stemmer = PorterStemmer()
# analyzer = TfidfVectorizer().build_analyzer()

# def stemmed_words(doc):
#     return (stemmer.stem(w) for w in analyzer(doc))

# tvec for reviews
tvec = TfidfVectorizer(stop_words = stopwords_review,
                       max_features = 10000,
                       norm = 'l2', 
                       strip_accents = 'unicode',
                       lowercase = True,
                       decode_error = 'replace',
                       ngram_range = (2, 2),
                       #analyzer = stemmed_words,
                       min_df = 0.1)

sparse_tvec = tvec.fit_transform(rv_final['review'])

In [14]:
tvec_df = pd.DataFrame(sparse_tvec.toarray(),
                       columns = tvec.get_feature_names())
tvec_df.head()

,abdominal pain,able eat,able function,able sleep,able work,absolutely effects,absolutely love,aches pains,acid reflux,actually feel,...,years used,years went,years worked,years works,years year,years years,yr old,yrs ago,yrs old,zero effects
0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2,0.000000,0.0,0.026229,0.0,0.080513,0.027129,0.0,0.028609,0.0,0.0,...,0.0,0.056397,0.048799,0.0265,0.000000,0.075514,0.044295,0.025659,0.022352,0.0
3,0.011974,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.027167,0.0,0.0,...,0.0,0.020083,0.000000,0.0000,0.006842,0.000000,0.010516,0.000000,0.005306,0.0
4,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [15]:
# get most frequent words
top_reviews_db = tvec_df.sum(axis = 0).sort_values(ascending = False)
top_reviews_db[:20]

blood pressure       64.009496
years ago            60.045829
started taking       59.619229
birth control        50.637931
twice day            43.918110
years old            43.775324
times day            41.960817
weight gain          40.114517
year old             38.568448
works great          35.782438
doctor prescribed    34.177500
feel better          33.710932
months ago           32.233764
dry mouth            30.527597
highly recommend     29.142022
stopped taking       28.932356
mood swings          27.987841
taking medication    27.530737
worked great         27.406940
went away            25.932124
dtype: float64

In [22]:
top_reviews_db = pd.DataFrame(top_reviews_db)
top_reviews_db.columns = ['tvec_count']
top_reviews_db.head()

,tvec_count
blood pressure,64.009496
years ago,60.045829
started taking,59.619229
birth control,50.637931
twice day,43.918110


In [23]:
# top_reviews_db.to_csv('nlp_reviews.csv')